In [12]:
import pandas as pd
from tqdm import tqdm

# 결측이 이전 시간대 사용하거나 이전 시간대가 없으면 전날..?


In [13]:
train =pd.read_csv('dataset/Train_data/Second_dataset.csv',low_memory=False)

In [15]:
numeric_weather_columns = [
    '출발현재기온값', '출발강수량', '출발적설량', '출발풍속',
    '도착현재기온값', '도착강수량', '도착적설량', '도착풍속',
    '출발시정값', '도착시정값'
]
string_weather_columns = ['출발현재일기내용', '도착현재일기내용']

# 숫자형 컬럼에 대해 같은 날, 같은 장소 기준으로 보간 적용
for col in tqdm(numeric_weather_columns):
    if col in train.columns:  # 데이터에 해당 컬럼이 있는지 확인
        train[col] = train.groupby(['집계일자', '출발영업소코드', '도착영업소코드'])[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

# 문자열 컬럼에 대해 같은 날, 같은 장소 기준으로 이전 시간대 값으로 채우기
for col in tqdm(string_weather_columns):
    if col in train.columns:
        train[col] = train.groupby(['집계일자', '출발영업소코드', '도착영업소코드'])[col].transform(lambda x: x.ffill())

100%|██████████| 2/2 [01:02<00:00, 31.46s/it]


In [16]:
train.to_csv('Second_NULL_test.csv',index=False,encoding='utf-8-sig')

train.isna().sum()

집계일자             0
집계시              0
출발영업소코드          0
도착영업소코드          0
TCS차종구분코드        0
통행시간             0
교통량              0
휴일 여부            0
출발현재일기내용     14993
출발시정값        12297
출발현재기온값      12297
출발강수량        12297
출발적설량        12297
출발풍속         12297
도착현재일기내용     15005
도착시정값        12297
도착현재기온값      12297
도착강수량        12297
도착적설량        12297
도착풍속         12297
dtype: int64

In [18]:
# 3. 남은 결측치에 대해 숫자형 컬럼은 각 일자의 평균값으로 채우기
for col in tqdm(numeric_weather_columns):
    if col in train.columns:
        train[col] = train.groupby('집계일자')[col].transform(
            lambda x: x.fillna(x.mean())
        )

# 4. 남은 결측치에 대해 문자열 컬럼은 각 일자의 최빈값으로 채우기
for col in tqdm(string_weather_columns):
    if col in train.columns:
        # Define a function to fill with mode or leave as NaN if mode is not available
        def fill_with_mode(series):
            mode = series.mode()
            if not mode.empty:
                return series.fillna(mode[0])
            else:
                return series

        train[col] = train.groupby('집계일자')[col].transform(fill_with_mode)

100%|██████████| 2/2 [00:01<00:00,  1.05it/s]


In [19]:
train.to_csv('Second_NULL_test_2.csv',index=False,encoding='utf-8-sig')

train.isna().sum()

집계일자             0
집계시              0
출발영업소코드          0
도착영업소코드          0
TCS차종구분코드        0
통행시간             0
교통량              0
휴일 여부            0
출발현재일기내용     12209
출발시정값        12209
출발현재기온값      12209
출발강수량        12209
출발적설량        12209
출발풍속         12209
도착현재일기내용     12209
도착시정값        12209
도착현재기온값      12209
도착강수량        12209
도착적설량        12209
도착풍속         12209
dtype: int64

In [20]:
missing_dates = train[train.isnull().any(axis=1)]['집계일자'].unique()

missing_dates

array([20231012], dtype=int64)